### Models Included
- SVM
- Bi-LSTM
- BiLSTM with Attention
- LSTM CNN

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, SimpleRNN, Flatten,\
Activation, RepeatVector, Permute, merge, Lambda
from keras_contrib.layers import CRF
import keras.optimizers as ko
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import keras
import random
import subprocess
from sklearn.utils import class_weight
import keras.backend as K
from keras.layers.convolutional import Conv1D, MaxPooling1D

In [ ]:
data = pd.read_csv('task2_typ.csv')
data = data.rename(columns={'id':'Sentence #'})
data = data.dropna(axis=0)
data = data.drop('Unnamed: 0',axis=1)

In [ ]:
data.groupby('type').count()

In [ ]:
sentences = []
labels = list(data['type']) 
typs = data['types'].values
x = data.apply(lambda l: sentences.append(l['sentence'].split(' ')),axis=1)

In [ ]:
words = []
for sent in sentences:
    for wrd in sent:
        words.append(wrd)
words = list(set(words))
tags = list(set(labels))

n_words = len(words)
n_tags = len(tags)

In [ ]:
len(typs)

In [ ]:
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
max_len = 30
X = [[word2idx[w] for w in s] for s in sentences]    
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [tag2idx[tg] for tg in labels]
# y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
y = [[y[i],typs[i]] for i in range(len(y))]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_train = np.array([lab[0] for lab in y_train])
typ_test = [lab[1] for lab in y_test]
y_test = np.array([lab[0] for lab  in y_test])

In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

vec_model = loadGloveModel('glove/glove.6B.200d.txt')

In [ ]:
emb_dim = len(vec_model['the'])
embedding_matrix = np.zeros((len(word2idx) + 1, emb_dim))

for word, i in word2idx.items():
    if word not in vec_model:
        continue
    embedding_vector = vec_model[word]
    embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

In [ ]:
def full_results(y_test,pred):
    
    report = classification_report(y_test,pred.flatten('F'),output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    
    inc_test = []
    exc_test = []
    inc_pred = []
    exc_pred = []
    for i,t in enumerate(typ_test):
        if t == 'I':
            inc_test.append(y_test[i])
            inc_pred.append(pred[i])
        else:
            exc_test.append(y_test[i])
            exc_pred.append(pred[i])
    print("For inclusions")
    report = classification_report(inc_test,inc_pred,output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    print("For exclusions")
    report = classification_report(exc_test,exc_pred,output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    return

def return_report(model,epochs):
    y_flat = list(np.argmax(y_test,1).flatten('F'))
#     class_weights = class_weight.compute_class_weight('balanced',y_flat)
#     class_weights = [1,10,20,10,20]
    model.fit(X_train,y_train,epochs=epochs,verbose=1)
    out = model.predict(X_test)
    pred = np.argmax(out,1)
    full_results(y_flat,pred)
    return pred


In [ ]:
def get_bilstm_lstm_model():
    
    input = Input(shape=(max_len,))

    # Add Embedding layer
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                  input_length=max_len, weights=[embedding_matrix],trainable=False)(input)

    # Add bidirectional LSTM
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.1))(model)    
    
    model = TimeDistributed(Dense(100,activation='relu'))(model)
    model = Flatten()(model)
    model = Dense(100,activation='relu')(model)
    # Add timeDistributed Layer
    out = Dense(n_tags, activation="softmax")(model)

    #Optimiser 
    adam = ko.Adam(lr=0.0007)

    # Compile model
    model = Model(input, out)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.summary()
    
    return model

def lstm_attention():
    
    input = Input(shape=(max_len,))

    # Add Embedding layer
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                  input_length=max_len, weights=[embedding_matrix],trainable=False)(input)

    # Add bidirectional LSTM
    model = Bidirectional(LSTM(units=emb_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.1))(model)

    attention = TimeDistributed(Dense(1, activation='tanh'))(model) 
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(2*emb_dim)(attention)
    attention = Permute([2, 1])(attention)

    # apply the attention
    sent_representation = merge.multiply([model, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=1))(sent_representation)
    probabilities = Dense(3, activation='softmax')(sent_representation)
    
    # Add timeDistributed Layer
    out = Dense(n_tags, activation="softmax")(sent_representation)

    #Optimiser 
    adam = ko.Adam(lr=0.0007)

    # Compile model
    model = Model(input, out)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    model.summary()
    
    return model

def lstm_cnn():
    input = Input(shape=(max_len,))

    # Add Embedding layer
    model = Embedding(input_dim=n_words + 1, output_dim=emb_dim,
                  input_length=max_len, weights=[embedding_matrix],trainable=False)(input)
    model = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu')(model)
    model = MaxPooling1D(pool_size=2)(model)
    model = LSTM(100,dropout=0.2,recurrent_dropout=0.2)(model)
    out = Dense(n_tags, activation='softmax')(model)
    model = Model(input,out)
    adam = ko.Adam(lr=0.0007)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    print(model.summary())
    
    return model


In [ ]:
bilistm = get_bilstm_lstm_model()
pred = return_report(bilistm,25)

In [ ]:
attn = lstm_attention()
preds = return_report(attn,50)

In [ ]:
lstmcnn = lstm_cnn()
preds = return_report(lstmcnn,50)

In [ ]:
bilistm.save('task2_models/bi-lstm/model')

### Simple Classifiers

In [ ]:
tag2idx = {t: i for i, t in enumerate(tags)}
max_len = 30
emb_dim = len(vec_model['the'])
X = []
y = []

for sent in sentences:
    vec = np.zeros((30,emb_dim))
    for i, word in enumerate(sent):
        if word in vec_model:
            vec[i,:] = vec_model[word]
    X.append(vec)
X = np.array(X)
y = np.array([tag2idx[tg] for tg in labels])
y = [[y[i],typs[i]] for i in range(len(y))]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = np.array([lab[0] for lab in y_train])
typ_test = [lab[1] for lab in y_test]
y_test = np.array([lab[0] for lab  in y_test])
X_train = X_train.reshape((X_train.shape[0],max_len*emb_dim))
X_test = X_test.reshape((X_test.shape[0],max_len*emb_dim))

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC(kernel='poly')
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
full_results(y_test,pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier